In [1]:
import numpy as np
from scipy import io as sio
from sklearn.covariance import OAS
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from matplotlib import pyplot as plt
from loader import get_dataloader

In [14]:
### TO COMPLETE ###
brainGraphPath = './neuro/graph.mat'
IBCPath = '/bigdisk2/nilearn_data/neurovault/collection_6618/'
splitDir = './neuro'

# Useful functions

### Data loader 

In [3]:
def generate_data_loader(n_shot, dataset, n_query=15, n_way=2):
    """
    Parameters:
        n_shot  --  number of training examples per class.
        dataset -- 'train', 'val' or 'test'. Split used to generate the few-shot problems.
        n_query  --  number of test examples per class.
        n_way  -- number of test examples per class.
    """
    sampler_infos = [1, n_way, n_shot, n_query]
    data_loader = get_dataloader(dataset, IBCPath, splitDir, True, sampler_infos=sampler_infos)
    return data_loader, sampler_infos


def brain_graph():
    """Return the adjacency matrix of the structural graph of the brain.
    """
    return sio.loadmat(brainGraphPath)['SC_avg56']

## Classification

In [4]:
def classifier(name):
    if name == 'LDA':
        oa = OAS(store_precision=False, assume_centered=False)
        clf = LinearDiscriminantAnalysis(solver='lsqr', covariance_estimator=oa)
    elif name == 'NCM':
        clf = NearestCentroid()
    elif name == 'LR':
        clf = LogisticRegression(random_state=0, max_iter=500)
    return clf

In [5]:
def few_shot_evaluation(data_loader, sampler_infos, n_task, weights=None, v=None, clf_name=None, show=True):
    """ Return the average accuracy on few-shot tasks.
    
    Parameters:
        data_loader  --  data_loader whose each batch contains a few-shot problem.
        sampler_infos  --  list containing [1, n_way, n_shot, n_query], the same as the one used to generate the data_loader.
        n_task  --  number of few-shot problems on which the results are averaged.
        weights  --  np.array containing coefficients used to normalize the features of the data samples.
        v  --  new basis on which the features of the data samples are projected.
        clf_name --  'LDA', 'NCM' or 'LR'.
    """
    acc_per_task = []
    for i in range(n_task):
        if show:
            print(i, end='\r')
        acc = task_evaluation(data_loader, sampler_infos, weights, v, clf_name)
        acc_per_task.append(acc)
    mean, conf = compute_confidence_interval(acc_per_task)
    if show:
        print('The baseline has an average accuracy of {:.2f}% over {} tasks with 95% confidence interval {:.2f}.'.format(mean*100, n_task, conf*100))
    return mean, conf

In [6]:
def task_evaluation(data_loader, sampler_infos, weights, v, clf_name):
    """
    Return the average accuracy on a few-shot task.
    A task contains training samples with known labels and query (test)
    samples. The accuracy is the number of times we correctly
    predict the labels of the query samples.
    """
    n_way = sampler_infos[1]
    n_shot = sampler_infos[2]
    n_query = sampler_infos[3]
    
    # Iterate over the number of episodes. (Here 1.)
    for x, y in data_loader: 
        assert x.shape[0] == n_way * (n_shot + n_query)  ## it is not the case if the data do not contain enough samples per class
        # Preprocess the data samples.
        x = x.view(x.shape[0], -1).numpy()
        if v is not None:
            assert weights is not None
            # Projection in the spectral space
            x = projection(x, v)
            # Optimize
            x = x * weights

        # Split the data into training samples and query samples.
        # Be careful: the data have to be sorted by split (train/query) and by classes.
        X_train = x[:n_way*n_shot]
        X_test = x[n_way*n_shot:]
        y_train = y[:n_way*n_shot]
        y_test = y[n_way*n_shot:]
        del x, y

        # Classify
        clf = classifier(clf_name)
        clf.fit(X_train, y_train)
        acc = clf.score(X_test, y_test)

    return acc


def projection(data, basis):
    """Return data projected into the vectors of basis.
    
    Parameters:
        data  --  matrix of shape [number of samples, length]
        basis  --  matrix of shape [length, number of vectors]
    """
    return np.matmul(data, basis)

In [7]:
def compute_confidence_interval(data):
    """
    Compute the mean and the 95% confidence interval of the values in data.
    """
    data = 1.0 * np.array(data)
    m = np.mean(data)
    std = np.std(data)
    conf = 1.96 * (std / np.sqrt(len(data)))
    return m, conf

## Our optimization method

In [8]:
def normalized_graph(GSO):
    degree = np.sum(GSO, axis=1)
    degree = degree ** (-1/2)
    degree = np.diag(degree)
    GSO = np.matmul(np.matmul(degree, GSO), degree)
    GSO = (GSO + GSO.T) / 2
    return GSO

def graph_fourier_transform(GSO):
    """Return the eigenvectors and eigenvalues of the graph shift operator GSO (e.g. adjacency matrix).
    """
    # Check whether the GSO is symmetric.
    assert (GSO == GSO.T).all()
    # Compute eigenvalues w and eigenvectors v.
    # The eigenvalues in w are sorted in ascending order.
    # v[:, i] is the normalized eigenvector corresponding to the eigenvalue w[i].
    w, v = np.linalg.eigh(GSO)
    return w, v

def optimal_weights(w, sigma):
    w = np.sqrt(w ** 2 + sigma ** 2)
    return 1 / w

# Results

In [9]:
# Setting
n_task = 10000
n_shot = 5
n_way = 5
n_query = 15
data_loader, sampler_infos = generate_data_loader(n_shot, 'test', n_query, n_way)

In [10]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, clf_name="NCM", show=True)

The baseline has an average accuracy of 74.18% over 1000 tasks with 95% confidence interval 0.59.


In [11]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, clf_name="LR", show=True)

The baseline has an average accuracy of 78.25% over 1000 tasks with 95% confidence interval 0.62.


In [12]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, clf_name="LDA", show=True)

The baseline has an average accuracy of 79.89% over 1000 tasks with 95% confidence interval 0.62.


In [15]:
# Graph used for our model
GSO = brain_graph()
GSO = normalized_graph(GSO)

# Our optimization
sigma = 0.5
w, v = graph_fourier_transform(GSO)
coefs = optimal_weights(w, sigma)

In [16]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, weights=coefs, v=v, clf_name="NCM", show=True)

The baseline has an average accuracy of 76.98% over 1000 tasks with 95% confidence interval 0.62.


In [17]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, weights=coefs, v=v, clf_name="LR", show=True)

The baseline has an average accuracy of 79.46% over 1000 tasks with 95% confidence interval 0.64.


In [18]:
mean, conf = few_shot_evaluation(data_loader, sampler_infos, n_task, weights=coefs, v=v, clf_name="LDA", show=True)

The baseline has an average accuracy of 79.46% over 1000 tasks with 95% confidence interval 0.64.


# Plot the graph

In [ ]:
# fig = plt.figure(figsize=(11, 10))
# ax = plt.axes()
# cmap = 'magma_r'
# cax = ax.matshow(GSO, cmap=cmap)
# ax.tick_params(axis='both', which='major', labelsize=20)
# # Add colorbar
# cbar = fig.colorbar(cax, ticks=[0.0005, 0.10], extend='max', shrink=.85)
# cax.set_clim(0, 0.1)
# # Vertically oriented colorbar
# cbar.ax.set_yticklabels(['0', '0.10'], fontsize=20)
# plt.subplots_adjust(left=.3, top=.99, right=.99, bottom=.275)
# plt.savefig('normalized_adjacency.png', dpi=1200, bbox_inches='tight')